In [43]:
from PyPDF2 import PdfReader

def extract_bookmarks(pdf_path):
    reader = PdfReader(pdf_path)
    bookmarks = reader.outline

    def recurse_bookmarks(bookmarks, depth=0):
        global chapter_num
        global chapter_list
        global cur_part
        for bkmk in bookmarks:
            if isinstance(bkmk, list) and depth < 2:
                recurse_bookmarks(bkmk, depth + 1)
            else:
                try:
                    title = bkmk['/Title']
                    page = reader.get_destination_page_number(bkmk)
                except KeyError:
                    raise ValueError('Bookmark is missing a title or page number')
                if depth == 0:
                    prefix = " Part "
                    cur_part = '[' + title + ']'
                    # cur_title = prefix + f'{title}, P:{page + 1}'
                    # chapter_list.append((cur_title, page + 1))
                    # print(cur_part)
                elif depth == 1:
                    prefix = cur_part
                    cur_title = f'{chapter_num} {title} ' + prefix + f' P:{page + 1}'
                    chapter_list.append((cur_title, page + 1))
                    # print(cur_title)
                    chapter_num += 1

    recurse_bookmarks(bookmarks)

cur_part = '-'
chapter_num = 2
chapter_list = []

pdf_path = 'Computer Vision -  Models, Learning, and Inference_compressed.pdf'
extract_bookmarks(pdf_path)

chapter_list = chapter_list[:-2]
chapter_list

[('2 Introduction to probability [I Probability] P:27', 27),
 ('3 Common probability distributions [I Probability] P:37', 37),
 ('4 Fitting probability models [I Probability] P:51', 51),
 ('5 The normal distribution [I Probability] P:71', 71),
 ('6 Learning and inference in vision [II Machine learning for machine vision] P:85',
  85),
 ('7 Modeling complex data densities [II Machine learning for machine vision] P:103',
  103),
 ('8 Regression models [II Machine learning for machine vision] P:145', 145),
 ('9 Classification models [II Machine learning for machine vision] P:173',
  173),
 ('10 Graphical models [III Connecting local models] P:219', 219),
 ('11 Models for chains and trees [III Connecting local models] P:245', 245),
 ('12 Models for grids [III Connecting local models] P:281', 281),
 ('13 Image preprocessing and feature extraction [IV Preprocessing] P:327',
  327),
 ('14 The pinhole camera [V Models for geometry] P:361', 361),
 ('15 Models for transformations [V Models for g

In [44]:
from PyPDF2 import PdfReader, PdfWriter

def split_pdf_by_chapters(pdf_path, chapters):
    reader = PdfReader(pdf_path)

    for i in range(len(chapters)):
        chapter_title, start_page = chapters[i]
        # Determine the end page for this chapter
        if i < len(chapters) - 1:
            end_page = chapters[i + 1][1] - 1
        else:
            end_page = len(reader.pages)

        writer = PdfWriter()
        # Add pages for the current chapter
        for page_num in range(start_page - 1, end_page):
            writer.add_page(reader.pages[page_num])

        # Output the chapter PDF
        output_filename = f"{chapter_title.strip().replace(' ', '_').replace(':', '').replace('[', '').replace(']', '')}.pdf"
        with open(output_filename, "wb") as output_pdf:
            writer.write(output_pdf)

split_pdf_by_chapters(pdf_path, chapter_list)
